In [ ]:
# Install dependencies from GitHub repository
!pip install git+https://github.com/jcanizalez/pytorch-time-series-forecasting.git

## 1. Load the Data

Read the sales data from the CSV file.

In [ ]:
import pandas as pd

# Download the sales data directly from GitHub
url = 'https://raw.githubusercontent.com/jcanizalez/pytorch-time-series-forecasting/main/data/sales.csv'
df = pd.read_csv(url)

# Preview the first five rows
df.head()